In [69]:
abs(-2)

2

In [19]:
import numpy as np
from data_augmentation import DataAugmentor_v1

da = DataAugmentor_v1()
a = np.array([[1,2,3],
        [4,5,6],
        [7,8,9]])

a = np.array([[]])

obs = {'board': a,
        'bomb_blast_strength': a, 
        'bomb_life': a,
        'bomb_moving_direction': a,
        'flame_life': a,
        'position': (0,0)}

test = da.augment(obs, 5, 3.0, obs, True)

test

[({'board': array([], shape=(0, 1), dtype=float64),
   'bomb_blast_strength': array([], shape=(0, 1), dtype=float64),
   'bomb_life': array([], shape=(0, 1), dtype=float64),
   'bomb_moving_direction': array([], shape=(0, 1), dtype=float64),
   'flame_life': array([], shape=(0, 1), dtype=float64),
   'position': (10, 0)},
  5,
  3.0,
  {'board': array([], shape=(0, 1), dtype=float64),
   'bomb_blast_strength': array([], shape=(0, 1), dtype=float64),
   'bomb_life': array([], shape=(0, 1), dtype=float64),
   'bomb_moving_direction': array([], shape=(0, 1), dtype=float64),
   'flame_life': array([], shape=(0, 1), dtype=float64),
   'position': (10, 0)},
  True),
 ({'board': array([], shape=(1, 0), dtype=float64),
   'bomb_blast_strength': array([], shape=(1, 0), dtype=float64),
   'bomb_life': array([], shape=(1, 0), dtype=float64),
   'bomb_moving_direction': array([], shape=(1, 0), dtype=float64),
   'flame_life': array([], shape=(1, 0), dtype=float64),
   'position': (10, 10)},
  5,
 

In [4]:
from pommerman.constants import Action
print(Action.Up.value, Action.Down.value, Action.Left.value, Action.Right.value, Action.Stop.value, Action.Bomb.value)

1 2 3 4 0 5


In [21]:
from pommerman.constants import Action

Action.Up.value
    
Action.Left.value

3

In [67]:
import numpy as np

def init_agent_list(agent1, agent2, policy1, policy2, enemy, setposition=False):
        '''
        Helper method for creating agent_list
        :param agent1: string identifying agent1
        :param agent2: string identifying agent2
        :param policy: policy a train agent follows
        :param setPosition: whether we initialize agent1 always on top left or randomly
        :return: agent indexes, igent ids on board observation and agent list of agent objects
        '''
        agent_list = [None] * 4
        agent_ind = 0 if setposition else np.random.randint(2)
        for i in range(0, 4):   

            if i == agent_ind:
                agent_str = agent1 
            elif i == agent_ind + 2:
                agent_str = agent2
            else:
                agent_str = enemy

            if agent_str.startswith('static'):
                _, action = agent_str.split(':')
                agent_list[i] = 'static' + str(action)
            elif agent_str == 'train' and i == agent_ind:
                agent_list[i] = 'train' + policy1
            elif agent_str == 'train' and i == agent_ind + 2:
                agent_list[i] = 'train' + policy2
            elif agent_str == 'smart_random':
                agent_list[i] = SmartRandomAgent()
            elif agent_str == 'simple':
                agent_list[i] = SimpleAgent()
            else:
                print('unsupported opponent type!')
                sys.exit(1)
        if 4 == 2:
            agent_inds = [0 + agent_ind]
            agent_ids = [10 + agent_ind]
        else:
            agent_inds = [0 + agent_ind, 2 + agent_ind]
            agent_ids = [10 + agent_ind, 12 + agent_ind]
        return agent_inds, agent_ids, agent_list


In [107]:
np.random.randint(2)

1

In [80]:
init_agent_list('train', 'train', '1', '2', 'static:2')

([0, 2], [10, 12], ['train1', 'static2', 'train2', 'static2'])

In [1]:
# solve error #15
import os    
os.environ['KMP_DUPLICATE_LIB_OK']='True'

NameError: name 'policy' is not defined

In [20]:
import os
import sys
import numpy as np
import random

import pommerman
from pommerman.agents import SimpleAgent, RandomAgent, PlayerAgent, BaseAgent
from pommerman.configs import ffa_v0_fast_env,  one_vs_one_env, team_competition_env
from pommerman.envs.v0 import Pomme
from pommerman.characters import Bomber
from pommerman import utility

from agents.static_agent import StaticAgent
from agents.train_agent import TrainAgent

from datetime import datetime
import os
import sys
import gym
import torch
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import params as p

from models import Pommer_Q
from data_generator import DataGeneratorPommerman
from dqn import DQN
from util.data import transform_observation, transform_observation_centralized
import params as p
from util.rewards import woods_close_to_bomb_reward

In [23]:
# model_dir1 = "../../data/tensorboard/20220120T172103/4491"
# model_dir2 = "../../data/tensorboard/20220120T172103/4492"

q = Pommer_Q(p.p_observable, transform_observation_centralized)
q_target = Pommer_Q(p.p_observable, transform_observation_centralized)

# q.load_state_dict(torch.load(model_dir1))
# q_target.load_state_dict(torch.load(model_dir1))
algo1 = DQN(q, q_target)
policy1 = algo1.get_policy()


# q.load_state_dict(torch.load(model_dir1))
# q_target.load_state_dict(torch.load(model_dir1))
algo2 = DQN(q, q_target)
policy2 = algo1.get_policy()



agent_list = [
    TrainAgent(policy1),
    StaticAgent(0),
    TrainAgent(policy2),
    StaticAgent(0),
]

env = pommerman.make('PommeTeamCompetition-v0', agent_list)

# # Seed and reset the environment
# env.seed(0)
# obs = env.reset()

# # Run the random agents until we're done
# done = False
# while not done:
#     env.render()
#     actions = env.act(obs)
#     obs, reward, done, info = env.step(actions)
#     print()
# env.render(close=True)
# env.close()

# print(info)

In [27]:
algo1.q_network.get_transformer()

<function models.Pommer_Q.get_transformer.<locals>.transformer(obs:dict) -> list>

## Test random agent with filter

In [40]:
import action_prune

class SmartRandomAgent(BaseAgent):
    """ random with filtered actions"""
    def act(self, obs, action_space):
        valid_actions=action_prune.get_filtered_actions(obs)
        if len(valid_actions) ==0:
            valid_actions.append(Action.Stop.value)
        return random.choice(valid_actions)

    def episode_end(self,reward):
        pass

NameError: name 'BaseAgent' is not defined

In [47]:
env.close()